In [18]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import glob
import h5py
import multiprocessing as mp

def ls2sol(ls):
    N_s = 668.6
    ls_peri = 250.99
    t_peri = 485.35
    a = 1.52368
    e = 0.09340
    epsilon = 25.1919
    if (ls == 0).any():
        ls = .01
    nu = np.radians(ls) + 1.90258
    E = np.arctan((np.tan(nu/2))/(np.sqrt((1+e)/(1-e))))*2
    M = E - e*np.sin(E)
    Ds = (M/(2*np.pi))*N_s + t_peri
    if (Ds < 0).any():
        Ds = Ds + N_s
    if (Ds > N_s).any():
        Ds = Ds - N_s
    return(Ds)

In [40]:
def point_loop(file):
    sg = xr.open_dataset('StupidGridFull.nc', group='flux')
    ds = xr.open_dataset(file)
    lat = ds['lat'][0]
    lon = ds['lon'][0]
    G = np.zeros(len(ds['lon']))
    for ri in range(0,len(ds['lon'])):
        ls = ds['ls'][ri]
        hr = ds['hr'][ri]
        G[ri] = sg['flux_dw_sw'][lat,lon,ls,hr]
    lss = np.unique(ds['ls'])
    Z = 2.016
    n = 2
    F = 96485.33212
    sg = 0
    try:
        P = G[:, np.newaxis, np.newaxis] * ds['j2_etaPEC_H2_2bg'] * Z/(n*F)
        zz = np.zeros((len(lss),len(ds['j2-bg1']),len(ds['j2-bg2'])))
        for i in range(0,len(lss)):
            hr_int = np.where(ds['ls']==lss[i])
            x = np.array(ds['hr'][hr_int] * 2 * 1.02569 * 60 * 60)
            for j in range(0,len(ds['j2-bg1'])):
                for k in range(0,len(ds['j2-bg2'])):
                    y = P[:,j,k][hr_int]
                    z = np.trapz(y,x=x)
                    zz[i,j,k] = z
        z = np.zeros((len(ds['j2-bg1']),len(ds['j2-bg2'])))
        x = ls2sol(lss*15) * 1.02569 * 24 * 60 * 60
        for j in range(0,len(ds['j2-bg1'])):
            for k in range(0,len(ds['j2-bg2'])):
                y = zz[:,j,k]
                z[j,k] = np.trapz(y,x=x)
        j2h2 = np.max(z)
        j2h2i = np.unravel_index(np.argmax(z),np.shape(z), order='C')

        h2 = j2h2 * (1/688)
        bg1 = np.array(ds['j2-bg1'][j2h2i[0]])
        bg2 = np.array(ds['j2-bg1'][j2h2i[1]])
        return([[lat,lon,0],[h2,bg1,bg2]])
    except:
        return([[lat,lon,1],[file]])

In [41]:
file = 'redsun_timeseries_00_10.nc'
results = [0,0]
results[0] = point_loop(file)

In [42]:
results

[[[<xarray.DataArray 'lat' ()>
   array(10),
   <xarray.DataArray 'lon' ()>
   array(0),
   1],
  ['redsun_timeseries_00_10.nc']],
 0]

In [21]:
results[1] = [0,0,1]
# print('CPUs: ' + str(mp.cpu_count()))
#
# pool = mp.Pool(processes=mp.cpu_count())
# results = pool.map(point_loop, [file for file in files])
# pool.close()

j2h2F = np.zeros((19,37))
j2h2i_bg1_F = np.zeros((19,37))
j2h2i_bg2_F = np.zeros((19,37))

for r in results:
    if r[0][2] == 0:
        lat = r[0][0]
        lon = r[0][1]
        j2h2F[lat,lon] = r[1][0]
        j2h2i_bg1_F[lat,lon] = r[1][1]
        j2h2i_bg2_F[lat,lon] = r[1][2]

lats = np.arange(-90,91,10)
lons = np.arange(-180,181,10)
test = xr.Dataset(
    {'j2h2': (('lon','lat'), j2h2F.transpose()),
    'j2h2-bg1': (('lon','lat'), j2h2i_bg1_F.transpose()),
    'j2h2-bg2': (('lon','lat'), j2h2i_bg2_F.transpose())},
    coords = {
        'lon': lons,
        'lat': lats,
    },
)
test['lat'].attrs['units'] = 'degrees_north'
test['lon'].attrs['units'] = 'degrees_east'
test['j2h2'].attrs['units'] = '(g)/(day*m^2)'
test['j2h2-bg1'].attrs['units'] = 'nm'
test['j2h2-bg2'].attrs['units'] = 'nm'

test['j2h2'] = test['j2h2'].where(test['j2h2'] != 0.0)
test['j2h2-bg1'] = test['j2h2-bg1'].where(test['j2h2-bg1'] != 0.0)
test['j2h2-bg2'] = test['j2h2-bg2'].where(test['j2h2-bg2'] != 0.0)


test.to_netcdf('j2h2_fix.nc')

TypeError: 'int' object is not subscriptable

In [22]:
results

[[[<xarray.DataArray 'lat' ()>
   array(0),
   <xarray.DataArray 'lon' ()>
   array(17),
   0],
  [36.23225871059901, array(1.51), array(1.62)]],
 0]

In [24]:
results

[[[<xarray.DataArray 'lat' ()>
   array(10),
   <xarray.DataArray 'lon' ()>
   array(0),
   0],
  [124.08336161963372, array(1.65), array(1.79)]],
 0]

In [30]:
ls2sol(np.unique(ds['ls'] * 15))

array([371.85494175, 397.20144076, 421.59393554, 445.2690196 ,
       468.48222144, 491.49867633, 640.04956588, 668.59949549])

In [33]:
ds['ls']

<xarray.DataArray 'ls' (record: 82)>
array([12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 24])
Dimensions without coordinates: record

In [34]:

ds

<xarray.Dataset>
Dimensions:                (j1-bg: 261, j2-bg1: 181, j2-bg2: 81, j3-bg1: 151, j3-bg2: 51, j3-bg3: 61, record: 82, wl: 1341)
Coordinates:
  * wl                     (wl) float64 300.5 301.5 302.5 ... 3.98e+03 4e+03
  * j1-bg                  (j1-bg) float64 0.4 0.41 0.42 0.43 ... 2.98 2.99 3.0
  * j2-bg1                 (j2-bg1) float64 1.2 1.21 1.22 1.23 ... 2.98 2.99 3.0
  * j2-bg2                 (j2-bg2) float64 0.4 0.41 0.42 0.43 ... 1.18 1.19 1.2
  * j3-bg1                 (j3-bg1) float64 1.5 1.51 1.52 1.53 ... 2.98 2.99 3.0
  * j3-bg2                 (j3-bg2) float64 1.0 1.01 1.02 1.03 ... 1.48 1.49 1.5
  * j3-bg3                 (j3-bg3) float64 0.4 0.41 0.42 0.43 ... 0.98 0.99 1.0
Dimensions without coordinates: record
Data variables:
    lat                    (record) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
    lon                    (record) int64 17 17 17 17 17 17 ... 17 17 17 17 17
    ls                     (record) int64 12 12 12 12 12 12 ... 23 23 23 23 24
    hr                     (record) int64 0 1 2 3 4 5 6 7 ... 5 6 7 9 10 11 12 0
    edir                   (record, wl) float64 ...
    edn                    (record, wl) float64 ...
    eup                    (record, wl) float64 ...
    eglo                   (record, wl) float64 ...
    mcd_lrt_error          (record) float64 ...
    j1_etaPV               (record, j1-bg) float64 ...
    j1_etaPEC_H2           (record, j1-bg) float64 ...
    j1_etaPEC_NH3          (record, j1-bg) float64 ...
    j1_etaPEC_AA           (record, j1-bg) float64 ...
    j1_max_etaPV           (record) float64 ...
    j1_max_etaPEC_H2       (record) float64 ...
    j1_max_etaPEC_NH3      (record) float64 ...
    j1_max_etaPEC_AA       (record) float64 ...
    j1_bg_PVmax            (record) float64 ...
    j1_bg_H2max            (record) float64 ...
    j1_bg_NH3max           (record) float64 ...
    j1_bg_AAmax            (record) float64 ...
    j2_etaPV_2bg           (record, j2-bg1, j2-bg2) float64 ...
    j2_etaPEC_H2_2bg       (record, j2-bg1, j2-bg2) float64 ...
    j2_etaPEC_NH3_2bg      (record, j2-bg1, j2-bg2) float64 ...
    j2_etaPEC_AA_2bg       (record, j2-bg1, j2-bg2) float64 ...
    j2_max_etaPV_2bg       (record) float64 ...
    j2_max_etaPEC_H2_2bg   (record) float64 ...
    j2_max_etaPEC_NH3_2bg  (record) float64 ...
    j2_max_etaPEC_AA_2bg   (record) float64 ...
    j2_bg1_PVmax           (record) float64 ...
    j2_bg2_PVmax           (record) float64 ...
    j2_bg1_H2max           (record) float64 ...
    j2_bg2_H2max           (record) float64 ...
    j2_bg1_NH3max          (record) float64 ...
    j2_bg2_NH3max          (record) float64 ...
    j2_bg1_AAmax           (record) float64 ...
    j2_bg2_AAmax           (record) float64 ...
    j3_etaPV_3bg           (record, j3-bg1, j3-bg2, j3-bg3) float64 ...
    j3_max_etaPV_3bg       (record) float64 ...
    j3_bg1max              (record) float64 ...
    j3_bg2max              (record) float64 ...
    j3_bg3max              (record) float64 ...
Attributes:
    history:                   Tue Jul 27 20:32:21 2021: ncecat ttlrecall_012...
    NCO:                       "4.6.4"
    nco_openmp_thread_number:  1

In [35]:

file = 'redsun_timeseries_00_10.nc'
sg = xr.open_dataset('StupidGridFull.nc', group='flux')
ds = xr.open_dataset(file)
lat = ds['lat'][0]
lon = ds['lon'][0]
G = np.zeros(len(ds['lon']))
for ri in range(0,len(ds['lon'])):
    ls = ds['ls'][ri]
    hr = ds['hr'][ri]
    G[ri] = sg['flux_dw_sw'][lat,lon,ls,hr]
lss = np.unique(ds['ls'])

In [39]:
lss

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])